In [ ]:
import io
import pandas as pd
import numpy as np

In [ ]:
job = "_sweed"
# fungi
#vcfs = ["Immitis.vcf","Gattii.vcf","pBrasiliensis.vcf","pJirovecii.vcf","Murina.vcf","Posadasii.vcf","histoplasmaG186ar.vcf","histoplasmaG217b.vcf","histoplasmaH88.vcf","histoplasmaNam1.vcf"]
#dirs = ["/pine/scr/m/a/mavenger/bwaFungifiles/immitisBams/mPileupSamtools","/pine/scr/m/a/mavenger/bwaFungifiles/gattiiBams/mpileupSamtools","/pine/scr/m/a/mavenger/bwaFungifiles/ParaBrasiliensis/samtoolsMpileup","/pine/scr/m/a/mavenger/bwaFungifiles/pneuJirovecii/MpileupSamtools","/pine/scr/m/a/mavenger/bwaFungifiles/Murina/samtools","/pine/scr/m/a/mavenger/bwaFungifiles/posadasiiBamZ/samtoolMpileup","/pine/scr/m/a/mavenger/bwaFungifiles/histoplasmaG186ar/samtoolsMPileup","/pine/scr/m/a/mavenger/bwaFungifiles/histoplasmaG217b/samtoolsMPileup","/pine/scr/m/a/mavenger/bwaFungifiles/histoplasmaH88/mPileupSamtools","/pine/scr/m/a/mavenger/bwaFungifiles/histoplasmaNam1/samtoolsMpileup"]

# flies
dirs = ["/pine/scr/m/a/mavenger/bwaDrosophilaFiles/mauritiana/samtoolsMpileup","/pine/scr/m/a/mavenger/bwaDrosophilaFiles/santomea/samtoolsMpile","/pine/scr/m/a/mavenger/bwaDrosophilaFiles/sechellia/samtoolsMPU","/pine/scr/m/a/mavenger/bwaDrosophilaFiles/yakuba/MpileupSamtools","/pine/scr/m/a/mavenger/bwaDrosophilaFiles/melonagater/samtoolMPile"]
vcfs = ["Mauritiana.vcf","Santomea.vcf","Sechellia.vcf","Yakuba.vcf","MeloMergeMpileup.vcf"]

for i in range(0,len(vcfs)):
    infile = vcfs[i]
    #sbatch submission script header
    script = 'script_' + infile + job + '.sh'
    outfile = io.open(script,'w', newline='\n') 
     
    outfile.write('#!/bin/bash\n\n#SBATCH --job-name='+infile+job+'\n')
    outfile.write('#SBATCH --mem=16G\n')
    outfile.write('#SBATCH --ntasks=4\n')
    outfile.write('#SBATCH --time=168:00:00 ## \n')
    outfile.write('#SBATCH -e '+infile+job+'_%A_%a.err ##error file if you want one (will be per job)\n')
    outfile.write('#SBATCH --mail-user=jmcgirr@email.unc.edu ##email you when job starts,ends,etc\n#SBATCH --mail-type=ALL\n\n')
    
    outfile.write('module load samtools \n')
    outfile.write('bcftools call --threads 4 -m -Ov -o /pine/scr/j/m/jmcgirr/matute/vcfs/fly/'+vcfs[i]+' '+dirs[i]+'/'+vcfs[i]+' \n')
    outfile.write('module load vcftools \n')
    outfile.write('vcftools --vcf /pine/scr/j/m/jmcgirr/matute/vcfs/fly/'+vcfs[i]+' --minQ 20 --maf 0.05 --max-missing 0.9 --remove-indels --min-alleles 2 --max-alleles 2 --recode --out /pine/scr/j/m/jmcgirr/matute/vcfs/fly/'+vcfs[i]+'_filtered_snps \n')
    outfile.write('sed \'s/,\*//g\' /pine/scr/j/m/jmcgirr/matute/vcfs/fly/'+vcfs[i]+'_filtered_snps.recode.vcf > /pine/scr/j/m/jmcgirr/matute/vcfs/fly/'+vcfs[i]+'_filtered_snps_sweed.vcf \n')
    outfile.write('/pine/scr/j/m/jmcgirr/matute/sweed/sweed-master/SweeD-P -input /pine/scr/j/m/jmcgirr/matute/vcfs/fly/'+vcfs[i]+'_filtered_snps_sweed.vcf -name '+vcfs[i]+'_grid1000 -strictPolymorphic -folded -grid 1000 -osfs '+vcfs[i]+'_sfs_grid1000 -threads 4 \n')
    outfile.write('sed \'/^#/ d\' /pine/scr/j/m/jmcgirr/matute/vcfs/fly/'+vcfs[i]+'_filtered_snps_sweed.vcf | awk \'{print $1}\' | cat -n | sort -k2 | uniq -f1 -d | sort -n | cut -f2- | nl > '+vcfs[i]+'_chrom_key.txt \n')
    
    #run sbatch submission 
    outfile.write('\n\n#command to run it on longleaf is sbatch '+script)
    outfile.close()

